In [1]:
from functools import wraps
import inspect
import sys

In [7]:
def log_cmd(path, mode='a'):
    def decorator(func):
        @wraps(func)
        def wrapped(*args, **kwargs):
            bound = inspect.signature(func).bind(*args, **kwargs)
            print(bound)
            if sys.argv[-1] not in ('--help', '-h'):
                res = '\npython'
                for arg in sys.argv:
                    pre = ' \\\n\t' if arg.startswith('-') else ' '
                    res += pre+arg
                print(res)
            return func(*args, **kwargs)
        return wrapped
    return decorator

In [38]:
@log_cmd('my_path')
def foo(a, b, *args, c=3, d='d', e='e', **kwargs):
    print('in foo')

In [39]:
foo(1, 'b')

<BoundArguments (a=1, b='b')>

python /Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/ipykernel_launcher.py \
	-f /Users/harrisonmamin/Library/Jupyter/runtime/kernel-b60d3cc6-5bf4-446d-8696-ae8ca72648a7.json
in foo


In [41]:
bound = inspect.signature(foo).bind('a', 'b2', 44, e='eee', g='g')
bound.apply_defaults()
fn_locals = bound.arguments
fn_locals.update(fn_locals.pop('kwargs'))
fn_locals

OrderedDict([('a', 'a'),
             ('b', 'b2'),
             ('args', (44,)),
             ('c', 3),
             ('d', 'd'),
             ('e', 'eee'),
             ('g', 'g')])

In [43]:
bound.kwargs

{'c': 3, 'd': 'd', 'e': 'eee'}

In [44]:
bound.args

('a', 'b2', 44)

In [42]:
bound.arguments

OrderedDict([('a', 'a'),
             ('b', 'b2'),
             ('args', (44,)),
             ('c', 3),
             ('d', 'd'),
             ('e', 'eee'),
             ('g', 'g')])

In [36]:
s = 'a={a}  b={b} c={c} d={d} e={e} g={g}'

In [45]:
s.format(**fn_locals)

'a=a  b=b2 c=3 d=d e=eee g=g'

In [29]:
def count_calls(func):
    @wraps(func)
    def wrapped(*args, **kwargs):
        calls = getattr(wrapped, 'calls', 0)
        wrapped.calls = calls + 1
        return func(*args, **kwargs)
    return wrapped

In [37]:
@count_calls
def foo(a):
    print(foo.calls)
    return a

In [38]:
foo(3)

1


3

In [39]:
foo.calls

1

In [40]:
foo.__wrapped__.calls

AttributeError: 'function' object has no attribute 'calls'

In [41]:
foo(4)

2


4

In [42]:
foo.calls

2

In [43]:
foo.__wrapped__.calls

AttributeError: 'function' object has no attribute 'calls'